In [20]:
from common.functions import *
import numpy as np
from pandarallel import pandarallel
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split

In [21]:
def estimations(running_times_test,
                running_times_train,
                cod_stop_from,
                cod_stop_to):

    running_times_test['trip_time_estimate'] = running_times_test.apply(get_running_time_estimate,
                                                                                 axis=1,
                                                                                 estimations=running_times_train).dt.round('1s')
    running_times_test['error'] = running_times_test['trip_time'].dt.total_seconds(
    ) - running_times_test['trip_time_estimate'].dt.total_seconds()
    running_times_test = running_times_test.query("error < 3600")

    return running_times_test

In [22]:
def get_running_time_estimate(row, estimations):
    cod_stop_from = row['cod_stop_from']
    cod_stop_to = row['cod_stop_to']
    cod_line = row['cod_line']
    cod_issue = row['cod_issue']
    departure_time = row['departure_time']
    arrival_time = row['arrival_time']

    estimation = estimations['trip_time'].mean()

    return estimation

In [23]:
cod_stop_from = '8_1336'
cod_stop_to = '8_06277'
cod_line = '8__658___'

In [24]:
running_times = filter_outliers(get_running_times(cod_stop_from, cod_stop_to))

if (cod_line):
    running_times = running_times[running_times['cod_line'] == cod_line]

running_times_train, running_times_test = train_test_split(running_times, test_size=0.25, random_state=42)

In [25]:
# # Initialization
# pandarallel.initialize(progress_bar=True)

In [26]:
plot_df = estimations(running_times_test, running_times_train, cod_stop_from, cod_stop_to)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
plot_graphs(plot_df)

In [28]:
get_metrics(plot_df)

MAE: 1.5764367816091958 minutes.
MAPE: 12.673317006357692 %.
RMSE: 1.9760963783560066 minutes.
                    trip_time trip_time_estimate       error
count                     116                116  116.000000
mean   0 days 00:13:21.103448    0 days 00:13:34  -12.896552
std    0 days 00:01:58.373644    0 days 00:00:00  118.373644
min           0 days 00:08:04    0 days 00:13:34 -330.000000
25%           0 days 00:12:02    0 days 00:13:34  -92.000000
50%    0 days 00:13:21.500000    0 days 00:13:34  -12.500000
75%    0 days 00:14:42.500000    0 days 00:13:34   68.500000
max           0 days 00:17:58    0 days 00:13:34  264.000000
